# Lesson 3: OpenAI Function Calling In LangChain

In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Pydantic Syntax

In [2]:
class User:
    def __init__(self,name:str,age:int,email:str):
        self.name=name
        self.age=age
        self.email=email
    

In [3]:
foo=User(name="Joe",age=32,email="joe@gmail.com")

In [4]:
foo.name

'Joe'

In [5]:
foo=User(name="Joe",age="bar",email="joe@gmail.com")
foo.age

'bar'

In [6]:
from typing import List
from pydantic import BaseModel, Field

class pUser(BaseModel):
    name:str
    age:int
    email:str

In [7]:
foo_p=pUser(name="Joe",age=32,email="joe@gmail.com")
foo_p.name

'Joe'

In [8]:
foo_p=pUser(name="Joe",age="bar",email="joe@gmail.com")
foo_p.age

ValidationError: 1 validation error for pUser
age
  value is not a valid integer (type=type_error.integer)

In [9]:
class Class(BaseModel):
    students:List[pUser]

In [10]:
obj=Class(students=[pUser(name="Joe",age=32,email="joe@gmail")])

obj

Class(students=[pUser(name='Joe', age=32, email='joe@gmail')])

# Pydantic to OpenAI function definition

In [11]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [12]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

weather_function=convert_pydantic_to_openai_function(WeatherSearch)

weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code',
    'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [13]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [14]:
convert_pydantic_to_openai_function(WeatherSearch1)

KeyError: 'description'

In [15]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [16]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch2',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code', 'type': 'string'}},
  'required': ['airport_code']}}

In [17]:
from langchain.chat_models import ChatOpenAI

model=ChatOpenAI()

In [18]:

model.invoke("what is the weather in SF today?", functions=[weather_function])


AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

In [19]:
model_with_function=model.bind(functions=[weather_function])

In [20]:
model_with_function.invoke("what is the weather in sf")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

# Forcing it to use a function

In [21]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [22]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

In [23]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

# Using in a chain

In [24]:
from langchain.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages([("system","You are a helpful assistant"),
("user","{input}")])

chain=prompt|model_with_function
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

# Using multiple functions

In [25]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [26]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [27]:
model_with_functions=model.bind(functions=functions)
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

In [28]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ArtistSearch', 'arguments': '{"artist_name":"Taylor Swift","n":3}'}})

In [29]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?')